# Course Summarization

## 0 General Concepts


deep learning is a subset of machine learning is a supset of artifical intelligece
artifcial intelligent is the sumulation of intellicene tought computer systems
machine learning creates ai thought computer algorythems and statistics
deep learning creates ai thought matmatical models call nural networks which in some respects simulatate the biological functions of the brain


# 1 Building the model
The arcuture of the model is dependent of the problem being solve. The problem i will be solving in this exable is a computer vission one

## 1.1 Creating the training and testing data

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
#!unzip -q drive/'My Drive'/UCFCourseWork/'CAP4630ArtificialIntelligence '/hw4/skin-cancer-malignant-vs-benign.zip
!ls
!cd test && ls

data  drive  sample_data  test	train
benign	malignant


In [0]:
train_dir = 'train'
validation_dir = 'test'

### Data Preprocessing

In [10]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

Using TensorFlow backend.


### Data generator

In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
batchSize = 32
numCategories = 2
classMode = "categorical"
# data augmentation

train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    color_mode="rgb",
    batch_size=batchSize,
    class_mode=classMode)

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    color_mode="rgb",
    batch_size=batchSize,
    class_mode=classMode)

Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


## 1.2 Building the model

### Pretraing Convlution model

In [12]:
from keras.applications import VGG16
from keras import layers
from keras import models
from keras import optimizers

conv_base = VGG16(
    weights='imagenet', 
    include_top=False, 
    input_shape=(150, 150, 3))

conv_base.trainable = False





58892288/58889256 [==============================] - 2s 0us/step








### Logical Protion of the neural network

In [0]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
# no dropout
model.add(layers.Dense(numCategories, activation='sigmoid'))

# 2 Complieing the model

Compiling the model has to key variables. loss which selects which loss function to use. and the learning rate which is how much the weight changes depending on the gradient. smaller lr that more time to train but are more precise

In [14]:
model.compile(
    loss='mean_squared_error',
    #
    # choose a smaller learning rate
    #
    optimizer=optimizers.RMSprop(lr=1e-5), 
    metrics=['acc'])

# 3 Training the model

When traing the net work the two variable that matter most are the epcoh and the steps per epoch. epoch is the number of timer the model is trained on the data set. higher numer means better results but may overfit. steps per epoch is the number of batches trained before the graident is computed and changing the weights of the model

In [15]:
epoch = 10
steps = 10
history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps,
    epochs=epoch,
    validation_data=validation_generator,
    validation_steps=steps)



Epoch 1/10
10/10 [==============================] - 145s 15s/step - loss: 0.2414 - acc: 0.5719 - val_loss: 0.2319 - val_acc: 0.6094
Epoch 2/10
10/10 [==============================] - 140s 14s/step - loss: 0.2282 - acc: 0.6562 - val_loss: 0.2218 - val_acc: 0.6781
Epoch 3/10
10/10 [==============================] - 139s 14s/step - loss: 0.2249 - acc: 0.6969 - val_loss: 0.2150 - val_acc: 0.7435
Epoch 4/10
10/10 [==============================] - 140s 14s/step - loss: 0.2231 - acc: 0.7094 - val_loss: 0.2139 - val_acc: 0.7312
Epoch 5/10
10/10 [==============================] - 138s 14s/step - loss: 0.2164 - acc: 0.6500 - val_loss: 0.2040 - val_acc: 0.6948
Epoch 6/10
10/10 [==============================] - 139s 14s/step - loss: 0.2123 - acc: 0.6875 - val_loss: 0.2065 - val_acc: 0.7250
Epoch 7/10
10/10 [==============================] - 134s 13s/step - loss: 0.2037 - acc: 0.7393 - val_loss: 0.1981 - val_acc: 0.7565
Epoch 8/10
10/10 [==============================] - 138s 14s/step - loss: 

# 4 Finetuning

Finetuning is the process of have a pretrain conv network frozen while the logical portion is train then the conv portion is unfrozen so that the conv portion is better tuned to the specific data it is being trained on

In [0]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [17]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=10,
    epochs=2,
    validation_data=validation_generator,
    validation_steps=10)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/2
10/10 [==============================] - 143s 14s/step - loss: 0.1828 - acc: 0.8250 - val_loss: 0.1837 - val_acc: 0.7781
Epoch 2/2
10/10 [==============================] - 136s 14s/step - loss: 0.1841 - acc: 0.8073 - val_loss: 0.1839 - val_acc: 0.7750


# 5 Saving and loading the network

## 5.1 Saving the model

In [0]:
# Save the model
model.save('path_to_my_model.h5')


## 5.2 loading the model

In [20]:
# Recreate the exact same model purely from the file
import keras
new_model = keras.models.load_model('path_to_my_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


# 6 Using the network

In [22]:
import cv2
from google.colab import files
files.upload() 


Saving 12.jpg to 12.jpg


{'12.jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xfe\x00&b8ca11a7-a643-11e2-a3f0-029118418759\xff\xed\x0e\xc2Photoshop 3.0\x008BIM\x04%\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x008BIM\x03\xed\x00\x00\x00\x00\x00\x10\x00H\x00\x00\x00\x01\x00\x01\x00H\x00\x00\x00\x01\x00\x018BIM\x04&\x00\x00\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00?\x80\x00\x008BIM\x04\r\x00\x00\x00\x00\x00\x04\x00\x00\x00\x1e8BIM\x04\x19\x00\x00\x00\x00\x00\x04\x00\x00\x00\x1e8BIM\x03\xf3\x00\x00\x00\x00\x00\t\x00\x00\x00\x00\x00\x00\x00\x00\x01\x008BIM\x04\n\x00\x00\x00\x00\x00\x01\x00\x008BIM\'\x10\x00\x00\x00\x00\x00\n\x00\x01\x00\x00\x00\x00\x00\x00\x00\x018BIM\x03\xf5\x00\x00\x00\x00\x00H\x00/ff\x00\x01\x00lff\x00\x06\x00\x00\x00\x00\x00\x01\x00/ff\x00\x01\x00\xa1\x99\x9a\x00\x06\x00\x00\x00\x00\x00\x01\x002\x00\x00\x00\x01\x00Z\x00\x00\x00\x06\x00\x00\x00\x00\x00\x01\x005\x00\x00\x00\x01\x00-\x00\x00\x00\x06\x00\x00\x00\x00\x

In [23]:
!ls


12.jpg	data  drive  path_to_my_model.h5  sample_data  test  train


In [36]:
im = cv2.imread("12.jpg")
im = im.reshape(465,620,3)
im = im[0:150, 0:150]
im = im.reshape(1,150,150,3)

predictions = new_model.predict(im)
index = 0
predictions[index]

array([0.9986418 , 0.01116672], dtype=float32)

The model Deturmined the mole is benign